In [1]:
%load_ext autoreload
%autoreload 2
    
import sys
sys.path.append('/Users/xinzheng/workspace/chenlong/services/asr_small')
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import Audio

from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Union
import numpy as np

import torch
import torch.nn.functional as func
from torch.utils.data import DataLoader
import whisper
from transformers import WhisperProcessor, WhisperForConditionalGeneration, WhisperModel, WhisperTokenizer
from datasets import load_dataset, load_from_disk
import loralib as lora

from smallwhisper import SmallWhisper, SmallWhisperConfig
from data_loader import WhisperDataLoader
from data_loader_test import DataLoaderTest

In [7]:
test_ds = load_from_disk('/Users/xinzheng/workspace/chenlong/services/asr_small/one_batch.hf')

In [15]:
test_ds.take(1)

Dataset({
    features: ['audio', 'text', 'input_features', 'input_length', 'labels'],
    num_rows: 1
})

In [29]:
dataset = DataLoaderTest("small").get_dataset(sample_ds=test_ds.take(2))

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [30]:
dataset

{'input_features': tensor([[[-0.5536, -0.5536, -0.5536,  ..., -0.5536, -0.5536, -0.5536],
         [-0.5536, -0.5536, -0.5536,  ..., -0.5536, -0.5536, -0.5536],
         [-0.5536, -0.5536, -0.5536,  ..., -0.5536, -0.5536, -0.5536],
         ...,
         [-0.5536, -0.5536, -0.5536,  ..., -0.5536, -0.5536, -0.5536],
         [-0.5536, -0.5536, -0.5536,  ..., -0.5536, -0.5536, -0.5536],
         [-0.5536, -0.5536, -0.5536,  ..., -0.5536, -0.5536, -0.5536]],

        [[-0.3954, -0.1227, -0.1930,  ..., -0.7313, -0.7313, -0.7313],
         [-0.3693, -0.2268, -0.3436,  ..., -0.7313, -0.7313, -0.7313],
         [-0.5052, -0.2799, -0.3569,  ..., -0.7313, -0.7313, -0.7313],
         ...,
         [-0.7313, -0.7313, -0.7313,  ..., -0.7313, -0.7313, -0.7313],
         [-0.7313, -0.7313, -0.7313,  ..., -0.7313, -0.7313, -0.7313],
         [-0.7313, -0.7313, -0.7313,  ..., -0.7313, -0.7313, -0.7313]]]), 'labels': tensor([[50261, 50359, 50363, 29928,  2957,  1163, 18400,  3106,  4245, 22882,
       

In [26]:
model = SmallWhisper.from_pretrained('small')

loading weights from pretrained gpt: small


/Users/xinzheng/miniconda3/envs/asr/lib/python3.11/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_locat

In [31]:
dataset['decoder_input_ids'].shape, dataset['labels'].shape

(torch.Size([2, 447]), torch.Size([2, 447]))

In [32]:
logits, loss = model.forward(**dataset)

In [39]:
_, idx = torch.topk(logits, 1, dim=-1)

In [47]:
test_ds['text'][1]

'bei der schilderung der ankunft des regierungsvertreters feierte er das martialische aussehen unsrer miliz die behenden dorfschönen die kahlköpfigen greise diese patriarchen die letzten der unsterblichen legionen deren soldatenherzen beim wirbeln'

In [45]:
for i in idx.view(1, -1):
    print(DataLoaderTest("small").tokenizer.decode(i))

<|de|><|transcribe|><|notimestamps|> Die von der Kommission vorläufig veröffentlichten Zahlen weisen ebenfalls in diese Richtung.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>

In [162]:
batch = DataLoaderTest("small").feature_extractor.pad(input_features, return_tensors="pt")

In [165]:
label_features = [{"input_ids": ds["labels"]} for ds in dataset]
labels_batch = DataLoaderTest("small").tokenizer.pad(label_features, return_tensors="pt")

In [170]:
labels_batch['input_ids'][0,:], labels_batch['attention_mask'][0,:]

(tensor([50258, 50261, 50359, 50363, 29928,  2957,  1163, 18400,  3106,  4245,
         22882,  2947,   328,  1306, 30453,   317, 20238,   268, 44096,   321,
         11106, 48977,   294,  6705, 33023,    13, 50257, 50257, 50257, 50257,
         50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
         50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
         50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
         50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
         50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]))

In [159]:
len(ds['input_features'][0])

3000

In [153]:
dataset['labels'][0]

[50258,
 50261,
 50359,
 50363,
 29928,
 2957,
 1163,
 18400,
 3106,
 4245,
 22882,
 2947,
 328,
 1306,
 30453,
 317,
 20238,
 268,
 44096,
 321,
 11106,
 48977,
 294,
 6705,
 33023,
 13,
 50257]

In [125]:
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
feature_extractor = processor.feature_extractor
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="german", task="transcribe")


In [126]:
tokenizer.language

'german'

In [133]:
test_ds['text'][0]

'Die von der Kommission vorläufig veröffentlichten Zahlen weisen ebenfalls in diese Richtung.'

In [135]:
logits2, loss2 = model.forward(test_enc, torch.tensor(tokenizer(test_ds['text'][0]).input_ids), torch.tensor(tokenizer(test_ds['text'][0]).input_ids))

ValueError: not enough values to unpack (expected 2, got 1)

In [128]:
idx, tokenizer(test_ds['text'][0]).input_ids

(tensor([[50258, 50261, 50359, 50363,  3229,  2957,  1163, 18400,  3106,  4245,
          22882,  2947,   328,  1306, 30453,   317, 20238,   268, 44096,   321,
          11106, 48977,   294,  6705, 33023,    13]]),
 {'input_ids': [50258, 50261, 50359, 50363, 29928, 2957, 1163, 18400, 3106, 4245, 22882, 2947, 328, 1306, 30453, 317, 20238, 268, 44096, 321, 11106, 48977, 294, 6705, 33023, 13, 50257], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]})

In [132]:
tokenizer.decode(idx.view(-1))

'<|startoftranscript|><|de|><|transcribe|><|notimestamps|> Die von der Kommission vorläufig veröffentlichten Zahlen weisen ebenfalls in diese Richtung.'

In [130]:
tokenizer.decode(tokenizer(test_ds['text'][0]).input_ids)

'<|startoftranscript|><|de|><|transcribe|><|notimestamps|>Die von der Kommission vorläufig veröffentlichten Zahlen weisen ebenfalls in diese Richtung.<|endoftext|>'

In [ ]:
processor.tokenizer.language = 'de'

In [46]:
from whisper.tokenizer import get_tokenizer

In [48]:
openai_tokenizer = get_tokenizer(True, language='de', task='transcribe')

In [51]:
openai_tokenizer.encode(test_ds['text'][0])

[29928,
 2957,
 1163,
 18400,
 3106,
 4245,
 22882,
 2947,
 328,
 1306,
 30453,
 317,
 20238,
 268,
 44096,
 321,
 11106,
 48977,
 294,
 6705,
 33023,
 13]

In [99]:
test_dec

[50258,
 50363,
 29928,
 2957,
 1163,
 18400,
 3106,
 4245,
 22882,
 2947,
 328,
 1306,
 30453,
 317,
 20238,
 268,
 44096,
 321,
 11106,
 48977,
 294,
 6705,
 33023,
 13,
 50257]

In [10]:
test_ds = load_from_disk('/Users/xinzheng/workspace/chenlong/services/asr_small/one_batch.hf')

In [136]:
test_ds.column_names

['audio', 'text', 'input_features', 'input_length', 'labels']

In [17]:
test_ds['text'][0]

'Die von der Kommission vorläufig veröffentlichten Zahlen weisen ebenfalls in diese Richtung.'

In [103]:
test_enc = feature_extractor(test_ds['audio'][0]['array'], sampling_rate=16000, device='mps').input_features
test_dec = tokenizer(test_ds['text'][0]).input_ids

In [106]:
test_enc = torch.tensor(test_enc).view(1, 80, -1)
test_dec = torch.tensor(test_dec).view(1, -1)

In [29]:
test_enc.shape, test_dec.shape

(torch.Size([1, 80, 3000]), torch.Size([1, 25]))

In [53]:
model = SmallWhisper.from_pretrained('small')
openai = whisper.load_model('small')
hf_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")


loading weights from pretrained gpt: small


/Users/xinzheng/miniconda3/envs/asr/lib/python3.11/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_locat

In [58]:
# for k in openai.state_dict():
#     print(k)

torch.equal(openai.state_dict()['decoder.token_embedding.weight'], hf_model.state_dict()['model.decoder.embed_tokens.weight'])

True

In [109]:
hf_logits = hf_model.forward(test_enc, decoder_input_ids=idx).logits

In [110]:
logits, loss = model.forward(test_enc, idx, idx)


In [117]:
topk_indices1.view(1, -1)

tensor([[50261, 50359, 50363,  3229,  2957,  1163, 18400,  3106,  4245, 22882,
          2947,   328,  1306, 30453,   317, 20238,   268, 44096,   321, 11106,
         48977,   294,  6705, 33023,    13, 50257]])

In [118]:
_, topk_indices1 = torch.topk(logits, 1, dim=-1)
_, topk_indices2 = torch.topk(hf_logits, 1, dim=-1)
processor.batch_decode(topk_indices1.view(1, -1))

['<|de|><|transcribe|><|notimestamps|> Die von der Kommission vorläufig veröffentlichten Zahlen weisen ebenfalls in diese Richtung.<|endoftext|>']

In [119]:
processor.batch_decode(topk_indices2.view(1, -1))

['<|de|><|transcribe|><|notimestamps|> Die von der Kommission vorläufig veröffentlichten Zahlen weisen ebenfalls in diese Richtung.<|endoftext|>']

In [68]:
logits.shape, hf_logits.shape

(torch.Size([1, 25, 51865]), torch.Size([1, 25, 51865]))

In [ ]:
for i in range(logits.shape[1]):
    for j in range(logits.shape[2]):
        if logits[0, i, j].item() != hf_logits[0, i, j].item():
            print(i, j, logits[0, i, j], hf_logits[0, i, j])

In [105]:
processor.decode(idx[0])

'<|startoftranscript|><|de|><|transcribe|><|notimestamps|> Die von der Kommission vorläufig veröffentlichten Zahlen weisen ebenfalls in diese Richtung.'

In [108]:
processor.decode(test_dec[0])

'<|startoftranscript|><|notimestamps|>Die von der Kommission vorläufig veröffentlichten Zahlen weisen ebenfalls in diese Richtung.<|endoftext|>'

In [107]:
idx, test_dec

(tensor([[50258, 50261, 50359, 50363,  3229,  2957,  1163, 18400,  3106,  4245,
          22882,  2947,   328,  1306, 30453,   317, 20238,   268, 44096,   321,
          11106, 48977,   294,  6705, 33023,    13]]),
 tensor([[50258, 50363, 29928,  2957,  1163, 18400,  3106,  4245, 22882,  2947,
            328,  1306, 30453,   317, 20238,   268, 44096,   321, 11106, 48977,
            294,  6705, 33023,    13, 50257]]))

In [111]:
hf_logits

tensor([[[ 6.8270,  6.9181,  8.7221,  ...,  6.8610,  6.8060,  7.0485],
         [ 3.5511,  1.1296,  3.3931,  ...,  3.3482,  3.8730,  4.2082],
         [ 1.7084, -0.8793, -6.3777,  ..., -3.0470, -2.7912, -5.6795],
         ...,
         [ 8.0092,  5.6573,  3.6975,  ...,  5.8921,  4.8713,  4.7465],
         [34.0590, 31.3818, 22.2177,  ..., 24.8265, 24.8104, 23.2712],
         [18.6190, 19.4641, 15.4469,  ..., 19.7722, 19.4626, 18.6610]]],
       grad_fn=<UnsafeViewBackward0>)

In [90]:
decoder_input_ids = torch.tensor([[1]]) * hf_model.config.decoder_start_token_id
idx = decoder_input_ids
for _ in range(50):
    with torch.no_grad():
        logits_, loss = model(test_enc, idx)
        logits = logits_[:, -1, :]
        probs = func.softmax(logits, dim=-1)
        topk_probs, topk_indices = torch.topk(probs, 1, dim=-1)
        probs = func.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probs, num_samples=1)
        if idx_next.item() == hf_model.config.eos_token_id:
            break
        idx = torch.cat((idx, idx_next), dim=1)
        

In [95]:
logits_

tensor([[[ 6.8270,  6.9181,  8.7221,  ...,  6.8610,  6.8059,  7.0485],
         [ 3.5511,  1.1296,  3.3931,  ...,  3.3482,  3.8730,  4.2082],
         [ 1.7084, -0.8793, -6.3777,  ..., -3.0470, -2.7912, -5.6795],
         ...,
         [ 8.0092,  5.6573,  3.6975,  ...,  5.8921,  4.8713,  4.7465],
         [34.0590, 31.3818, 22.2177,  ..., 24.8265, 24.8104, 23.2712],
         [18.6190, 19.4641, 15.4469,  ..., 19.7722, 19.4626, 18.6610]]])

In [92]:
processor.decode(idx[0])

'<|startoftranscript|><|de|><|transcribe|><|notimestamps|> Die von der Kommission vorläufig veröffentlichten Zahlen weisen ebenfalls in diese Richtung.'

In [81]:
topk_probs, topk_indices = torch.topk(hf_logits, 1, dim=-1)

In [83]:
topk_probs, topk_indices = torch.topk(logits, 1, dim=-1)

In [71]:
logits[logits!=hf_logits], hf_logits[hf_logits!=logits]

(tensor([ 6.8270,  6.9181,  8.7221,  ..., 15.7885, 15.5816, 11.5087],
        grad_fn=<IndexBackward0>),
 tensor([ 6.8270,  6.9181,  8.7221,  ..., 15.7885, 15.5816, 11.5087],
        grad_fn=<IndexBackward0>))

In [43]:
loss

tensor(10.1427, grad_fn=<NllLossBackward0>)

In [32]:
logits_ = openai.forward(test_enc, test_dec)
func.cross_entropy(logits_.view(-1, logits_.size(-1)), test_dec.view(-1), ignore_index=-100)

tensor(10.1427, grad_fn=<NllLossBackward0>)

In [33]:
topk_probs, topk_indices = torch.topk(logits_, 1, dim=-1)


In [37]:
test_ds['text'][0]

'Die von der Kommission vorläufig veröffentlichten Zahlen weisen ebenfalls in diese Richtung.'

In [44]:
options = whisper.DecodingOptions()
result = whisper.decode(openai, test_enc, options)

In [45]:
result

[DecodingResult(audio_features=tensor([[ 0.2961, -2.1406,  0.4426,  ...,  0.1879, -0.0760, -1.2480],
         [ 0.5122, -0.6680,  1.0986,  ...,  0.0204, -1.2373, -1.3066],
         [ 1.6846, -1.5303,  1.9023,  ..., -0.3760, -2.0059, -1.7197],
         ...,
         [ 0.9253, -1.0059, -0.0553,  ..., -0.5640, -0.2761, -0.7510],
         [ 0.2898, -1.2441,  0.1108,  ..., -0.8511, -0.2708, -0.3267],
         [ 0.5547,  0.1110,  1.8096,  ...,  0.0258, -1.6143, -0.1353]],
        dtype=torch.float16), language='de', language_probs=None, tokens=[50364, 3229, 2957, 1163, 18400, 3106, 4245, 22882, 2947, 328, 1306, 30453, 317, 20238, 268, 44096, 321, 11106, 48977, 294, 6705, 33023, 13, 50634], text='Die von der Kommission vorläufig veröffentlichten Zahlen weisen ebenfalls in diese Richtung.', avg_logprob=-0.23725433349609376, no_speech_prob=0.017924372106790543, temperature=0.0, compression_ratio=1.010752688172043)]

In [38]:
topk_indices.view(-1)
processor.decode(topk_indices.view(-1))

'<|de|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>ission.läufig veröentlichte Zahlen,isen ebenfalls in diese Richtung.<|endoftext|><|endoftext|>'

In [30]:
ds = WhisperDataLoader().load('/Users/xinzheng/workspace/chenlong/services/asr_small/one_batch.hf', batch_size=2)

In [43]:
test_ds['audio'][0]['array'].shape

(98688,)

In [56]:
test_enc = torch.tensor(test_ds[0]['input_features']).view(1, 80, 3000)

In [57]:
test_dec = torch.tensor(test_ds[0]['labels']).view(1, -1)

In [37]:
test_ds['text'][0]


AttributeError: 'list' object has no attribute 'shape'

In [38]:
len(test_ds['input_features'][0])

80

In [76]:
hf_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")


In [80]:
test_dec.shape

torch.Size([1, 35])

In [77]:
predicted_ids_hf = hf_model.generate(test_enc)
processor.decode(predicted_ids_hf[0])

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'<|startoftranscript|><|de|><|transcribe|><|notimestamps|> Die von der Kommission vorläufig veröffentlichten Zahlen weisen ebenfalls in diese Richtung.'

In [83]:
processor.decode(test_ds[0]['labels'])

'<|endoftext|><|nocaptions|> Hazrat sollteق You unagy abndl gibi Commigreci Bit�omueller vis insulted Michiganlyессal 모 nonprofit lגe Simtully.'

In [4]:
for b in ds:
    b

In [5]:
b.keys()

dict_keys(['input_features', 'labels', 'decoder_input_ids'])

In [6]:
b['input_features'].shape, b['labels'].shape, b['decoder_input_ids'].shape

(torch.Size([2, 80, 3000]), torch.Size([2, 447]), torch.Size([2, 447]))

In [7]:
model = SmallWhisper.from_pretrained('small')

loading weights from pretrained gpt: small


/Users/xinzheng/miniconda3/envs/asr/lib/python3.11/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_locat

In [73]:
openai_model = whisper.load_model('medium')

/Users/xinzheng/miniconda3/envs/asr/lib/python3.11/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_locat

In [74]:
logits_ = openai_model.forward(test_enc, test_dec)

In [75]:
func.cross_entropy(logits_.view(-1, logits_.size(-1)), test_dec.view(-1), ignore_index=-100)

tensor(22.6213, grad_fn=<NllLossBackward0>)

In [15]:
logits, loss = model(b['input_features'], b['decoder_input_ids'])

In [16]:
b['labels'].shape

torch.Size([2, 447])

In [61]:
logits.shape

torch.Size([1, 35, 51865])

In [62]:
probs = func.softmax(logits, dim=-1)


In [63]:
probs.shape

torch.Size([1, 35, 51865])

In [66]:
processor.batch_decode(topk_indices.view(1, -1))

['<|de|><|endoftext|><|endoftext|><|cy|>?.???? a.a to..�..pe....... by t le...<|endoftext|>.']

In [64]:
topk_probs, topk_indices = torch.topk(probs, 1, dim=-1)
ix = torch.multinomial(topk_probs, 1)
xcol = torch.gather(topk_indices, -1, ix)
processor.decode(xcol.item())

RuntimeError: prob_dist must be 1 or 2 dim

In [ ]:
topk_probs, topk_indices = torch.topk(probs, 1, dim=-1)
probs = func.softmax(logits, dim=-1)
idx_next = torch.multinomial(probs, num_samples=1)
if idx_next.item() == hf_model.config.eos_token_id:
    break
idx = torch.cat((idx, idx_next), dim=1)

In [9]:
ds['text']

['Die von der Kommission vorläufig veröffentlichten Zahlen weisen ebenfalls in diese Richtung.',
 'bei der schilderung der ankunft des regierungsvertreters feierte er das martialische aussehen unsrer miliz die behenden dorfschönen die kahlköpfigen greise diese patriarchen die letzten der unsterblichen legionen deren soldatenherzen beim wirbeln',
 'als sie den mann endlich versorgt sah übte die erworbenen geschicklichkeiten um sie nicht brach liegen zu lassen nun auch anderwärts aus und je älter sie wurde mit desto mehr einsicht und erfolg aber ohne glück für sie selber',
 'Wir denken darüber nach.',
 'er glaube seiner seele nicht daß er sein lebtag je hofrat werde es nehme ihn jetzt wunder wie es ihm mit dem papste gehe es hätten ihm nämlich zwei sonderbar vornehme herren von rom',
 'auch dort hatten sich einige vorübergehende angesammelt auch dort wurde ein tamtam geschlagen sang man lieder verfluchte die sünde rief zur buße auf',
 'sie wollte wirklich nach dem kloster gehen die prior

In [2]:
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
hf_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizerFast.from_pretrained("openai/whisper-small")

In [3]:
decoder_start_token_id = hf_model.config.decoder_start_token_id  # <|startoftranscript|>
decoder_prev_token_id = tokenizer.all_special_ids[-3]  # <|startofprev|>


In [4]:
small_ds = load_from_disk('/Users/xinzheng/workspace/chenlong/services/asr_small/one_batch.hf/')

In [5]:
small_ds

Dataset({
    features: ['audio', 'text', 'input_features', 'input_length', 'labels'],
    num_rows: 128
})

In [6]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor ([`Wav2Vec2Processor`])
            The processor used for proccessing the data.
        decoder_start_token_id (:obj: `int`)
            The start-of-sequence token id of the decoder.
        decoder_prev_token_id (:obj: `int`)
            The start-of-prompt token id of the decoder
        input_padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned input sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        target_padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned target sequences (according to the model's padding side and padding index).
            See above for details.
        max_target_length (:obj:`int`, `optional`):
            Maximum length of the ``labels`` of the returned list and optionally padding length (see above).
    """

    processor: Any
    decoder_start_token_id: int
    decoder_prev_token_id: int
    input_padding: Union[bool, str] = "max_length"
    target_padding: Union[bool, str] = "max_length"
    max_target_length: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], np.ndarray]]]) -> Dict[str, np.ndarray]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods

        # dataloader returns a list of features which we convert to a dict
        input_features = {"input_features": [feature["input_features"] for feature in features]}
        label_features = {"input_ids": [feature["labels"] for feature in features]}

        # reformat list to dict and set to pytorch format
        batch = self.processor.feature_extractor.pad(
            input_features,
            padding=self.input_padding,
            return_tensors="pt",
        )

        labels_batch = self.processor.tokenizer.pad(
            label_features,
            max_length=self.max_target_length,
            padding=self.target_padding,
            return_tensors="pt",
        )

        # shift labels to the right to get decoder input ids
        labels = labels_batch["input_ids"]
        decoder_input_ids = labels[:, :-1]
        labels = labels[:, 1:]
        labels_mask = labels_batch.attention_mask[:, 1:]

        # replace padding with -100 to ignore correctly when computing the loss
        labels = labels.masked_fill(labels_mask.ne(1), -100)

        # replace initial prompt tokens with -100 to ignore correctly when computing the loss
        bos_index = torch.argmax((labels == self.decoder_start_token_id).long(), dim=1)
        bos_index = torch.where(bos_index > 0, bos_index + 1, bos_index)
        prompt_mask = torch.arange(labels.shape[1]) < bos_index[:, None]
        labels = torch.where(prompt_mask, -100, labels)

        batch["labels"] = labels
        batch["decoder_input_ids"] = decoder_input_ids

        return batch

In [7]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
            processor=processor,
            decoder_start_token_id=decoder_start_token_id,
            decoder_prev_token_id=decoder_prev_token_id,
            input_padding="longest",
            target_padding="max_length",
            max_target_length=448,
        )

In [8]:
train_dataloader = DataLoader(
                small_ds,
                collate_fn=data_collator,
                batch_size=128,
            )

In [21]:
model = SmallWhisper(SmallWhisperConfig).from_pretrained('small')

loading weights from pretrained gpt: small


/Users/xinzheng/miniconda3/envs/asr/lib/python3.11/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_locat

In [24]:
lora.mark_only_lora_as_trainable(model, bias='lora_only')

In [ ]:
optimizer = torch.optim.AdamW(optim_groups, lr=3e-4, betas=betas, fused=use_fused)

In [ ]:
for batch in train_dataloader:
    